In [7]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [8]:
# Physical Constants
hbar = 1.055e-34      # Js
q = 1.602e-19         # C
m = 9.1e-31           # kg
m_dot = 0.2 * m       # kg  
bohr_radii = 5.29e-11 # m
a = 1e-10             # m

In [9]:
def VDQD(alpha, x1, x2, x):
    return alpha * min((x - x1) ** 2, (x - x2) ** 2)

def VFT(alpha, F, omega, x1, x2, x, t):
    return alpha * min((x - x1) ** 2, (x - x2) ** 2) + F * x * np.cos(omega * t)

In [10]:
@jit
def construct_H(U):
    return np.diag(on + U) + np.diag(off, 1) + np.diag(off, -1)

In [11]:
@jit
def compute_eigenvalue_eigenstates(U):
    H = construct_H(U)
    W, V = np.linalg.eigh(H)
    idx = W.argsort()[::1]
    W = W[idx]
    V = V[:, idx]
    return W, V

In [12]:
# Spatial Constants
Np = 700
X = a * np.linspace(-Np / 2, Np / 2, Np) / 1e-9  # nm
dx = (X[1] - X[0]) * 1e-9

t0 = hbar ** 2 / (2 * m_dot * (dx ** 2)) / q
on = 2.0 * t0 * np.ones(Np)
off = -t0 * np.ones(Np - 1)

alpha = 2.1e-5

In [13]:
# Temporal Constants
Nt = 100

In [27]:
index = 0
for R in [12.5, 15, 17.5,]:
    x1, x2 = R, -R
    U = np.array([VDQD(alpha, x1, x2, x) for x in X])
    W, V = compute_eigenvalue_eigenstates(U)
    
    omega = (W[1] - W[0]) / hbar * q
    T = np.linspace(0, 4 * np.pi / omega, Nt)         
    dt = T[1] - T[0]                                  
    
    for F in np.arange(0, 15, 0.5) * 1e-5:
        U = np.array([VFT(alpha, F, omega, x1, x2, x, 0) for x in X])
        W, V = compute_eigenvalue_eigenstates(U)

        psi = np.array(V[:, 0], dtype=np.complex128)
        H = construct_H(U).astype(np.complex128)

        p0 = np.array([np.abs(psi @ V[:, 0])])
        p1 = np.array([np.abs(psi @ V[:, 1])])

        for i in range(1, Nt):
            t = T[i]
            U_new = np.array([VFT(alpha, F, omega, x1, x2, x, t) for x in X])
            H_new = construct_H(U_new).astype(np.complex128)

            A = (np.identity(Np) + 1j * dt / (2 * hbar) * H_new * q)
            B = (np.identity(Np) - 1j * dt / (2 * hbar) * H * q) @ psi
            psi_new = np.linalg.solve(A, B)

            pr0 = np.abs(psi_new @ V[:, 0]) ** 2
            pr1 = np.abs(psi_new @ V[:, 1]) ** 2

            p0 = np.append(p0, pr0)
            p1 = np.append(p1, pr1)

            psi = psi_new
            H = H_new

        plt.figure(index)
        plt.plot(p0)
        plt.plot(p1)
        index += 1
        plt.savefig(f'R={R:.2f}+F={F}.png')
        plt.close()
        print(f'Did R={R:.2f}+F={F}')

Did R=12.50+F=0.0
Did R=12.50+F=5e-06
Did R=12.50+F=1e-05
Did R=12.50+F=1.5000000000000002e-05
Did R=12.50+F=2e-05
Did R=12.50+F=2.5e-05
Did R=12.50+F=3.0000000000000004e-05
Did R=12.50+F=3.5000000000000004e-05
Did R=12.50+F=4e-05
Did R=12.50+F=4.5e-05
Did R=12.50+F=5e-05
Did R=12.50+F=5.5e-05
Did R=12.50+F=6.000000000000001e-05
Did R=12.50+F=6.500000000000001e-05
Did R=12.50+F=7.000000000000001e-05
Did R=12.50+F=7.500000000000001e-05
Did R=12.50+F=8e-05
Did R=12.50+F=8.5e-05
Did R=12.50+F=9e-05
Did R=12.50+F=9.5e-05
Did R=12.50+F=0.0001
Did R=12.50+F=0.000105
Did R=12.50+F=0.00011
Did R=12.50+F=0.000115
Did R=12.50+F=0.00012000000000000002
Did R=12.50+F=0.000125
Did R=12.50+F=0.00013000000000000002
Did R=12.50+F=0.000135
Did R=12.50+F=0.00014000000000000001
Did R=12.50+F=0.000145
Did R=15.00+F=0.0
Did R=15.00+F=5e-06
Did R=15.00+F=1e-05
Did R=15.00+F=1.5000000000000002e-05
Did R=15.00+F=2e-05
Did R=15.00+F=2.5e-05
Did R=15.00+F=3.0000000000000004e-05
Did R=15.00+F=3.5000000000000004e-